In [1]:
!nvidia-smi

Mon Aug 18 08:04:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   61C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Install libraries we’ll use
!pip install -q ultralytics opencv-python onnx onnxruntime-gpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.2/283.2 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Robust installs into the active Python environment
import sys, subprocess, platform, os

def pip(*args):
    print(">>> pip", *args)
    subprocess.check_call([sys.executable, "-m", "pip", *args])

# Clean up any partial/broken TRT installs (ok if they weren't present)
try:
    pip("uninstall", "-y", "tensorrt", "tensorrt-cu12")
except Exception as e:
    print("Uninstall note:", e)

# Install core packages
pip("install", "-U", "ultralytics", "onnx>=1.12,<1.18", "onnxslim>=0.1.59", "onnxruntime-gpu")

# TensorRT wheels (try CUDA 12 build first, then generic)
trt_ok = False
for pkg in ("tensorrt-cu12", "tensorrt"):
    try:
        pip("install", "-U", "--extra-index-url", "https://pypi.nvidia.com", pkg)
        import importlib, tensorrt as trt
        print("TensorRT:", trt.__version__, f"({pkg})")
        trt_ok = True
        break
    except Exception as e:
        print(f"Install/import failed for {pkg} →", e)

if not trt_ok:
    raise SystemExit("❌ TensorRT wheel install failed. Switch Colab to Python 3.10/3.11 and GPU, then re-run.")

# Sanity prints
import torch, ultralytics, cv2
print("Python:", sys.version.split()[0])
print("Torch:", torch.__version__)
print("Ultralytics:", ultralytics.__version__)
print("OpenCV:", cv2.__version__)
!nvidia-smi

assert torch.cuda.is_available(), "Enable GPU in Colab: Runtime → Change runtime type → GPU"


Mounted at /content/drive
>>> pip uninstall -y tensorrt tensorrt-cu12
>>> pip install -U ultralytics onnx>=1.12,<1.18 onnxslim>=0.1.59 onnxruntime-gpu
>>> pip install -U --extra-index-url https://pypi.nvidia.com tensorrt-cu12
TensorRT: 10.13.2.6 (tensorrt-cu12)
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Python: 3.11.13
Torch: 2.8.0+cu128
Ultralytics: 8.3.179
OpenCV: 4.12.0
Mon Aug 18 08:17:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-

In [ ]:
from pathlib import Path
import yaml, glob, os


WEIGHTS_PT = "/content/drive/MyDrive/yolo11_custom_run/best.pt"
DATA_YAML  = "/content/drive/MyDrive/yolo11_custom_run/custom_dataset.yaml"


IMGSZ = 640
CONF  = 0.50
BATCH = 1                
WORKSPACE_GB = 4.0
CALIB_FRACTION = 1.0      

# Output folders (fast in /content; we’ll copy to Drive later)
OUT_DIR   = Path("/content/outs2"); OUT_DIR.mkdir(parents=True, exist_ok=True)
ENG_FP32  = OUT_DIR/"yolo11m_custom_fp32_b1_640.engine"
ENG_FP16  = OUT_DIR/"yolo11m_custom_fp16_b1_640.engine"
ENG_INT8  = OUT_DIR/"yolo11m_custom_int8_b1_640_frac1.0.engine" 

# Verify inputs
assert Path(WEIGHTS_PT).is_file(), f"Missing weights: {WEIGHTS_PT}"
assert Path(DATA_YAML).is_file(), f"Missing dataset yaml: {DATA_YAML}"

# Optional: quick dataset sanity
with open(DATA_YAML, "r") as f:
    cfg = yaml.safe_load(f)
root = Path(cfg.get("path", "."))
for split in ("train","val","test"):
    sp = cfg.get(split, None)
    assert sp is not None, f"'{split}' split missing in {DATA_YAML}"
    d = (root/sp).resolve()
    n_img = len([p for p in glob.glob(str(d/"**/*.*"), recursive=True)
                 if p.lower().endswith((".jpg",".jpeg",".png",".bmp",".tif",".tiff",".webp"))])
    n_lab = len([p for p in glob.glob(str((root/'labels'/Path(sp).name)/"**/*.txt"), recursive=True)])
    print(f"{split:5}: {n_img} images (labels count {n_lab} may differ depending on structure)")


train: 2515 images (labels count 0 may differ depending on structure)
val  : 1508 images (labels count 0 may differ depending on structure)
test : 1008 images (labels count 0 may differ depending on structure)


In [5]:
import time, shutil, glob

def find_newest_engine(since_time: float):
    paths = [p for p in glob.glob("**/*.engine", recursive=True)]
    paths = [(p, os.path.getmtime(p)) for p in paths if os.path.getmtime(p) >= since_time - 5]
    if not paths:
        paths = [(p, os.path.getmtime(p)) for p in glob.glob("**/*.engine", recursive=True)]
    if not paths:
        return None
    paths.sort(key=lambda x: x[1], reverse=True)
    return paths[0][0]

def export_engine(model, dest_path, *, half=False, int8=False, data_yaml=None,
                  imgsz=640, batch=1, workspace=4.0, fraction=None, dynamic=False, verbose=False):
    """
    Uses Ultralytics exporter to build a TensorRT engine.
    - half=True  → FP16
    - int8=True  → INT8 (requires data_yaml for calibrator; uses 'val' split)
    - batch=1    → static engine compatible with single-image inference loops
    """
    t0 = time.time()
    args = dict(format="engine", imgsz=imgsz, half=half, int8=int8,
                batch=batch, workspace=workspace, dynamic=dynamic, verbose=verbose)
    if int8:
        assert data_yaml is not None and Path(data_yaml).exists(), "INT8 export needs data=DATA_YAML"
        args.update(dict(data=str(data_yaml)))
        if fraction is not None:
            args.update(dict(fraction=float(fraction)))
    _ = model.export(**args)
    found = find_newest_engine(t0)
    assert found, "Engine not found after export."
    shutil.move(found, dest_path)
    kind = "FP16" if half else ("INT8" if int8 else "FP32")
    print(f"{kind} engine → {dest_path}")
    return str(dest_path)


In [6]:
from ultralytics import YOLO
model = YOLO(WEIGHTS_PT)

# FP32
export_engine(model, ENG_FP32, half=False, int8=False, imgsz=IMGSZ, batch=BATCH,
              workspace=WORKSPACE_GB, dynamic=False)

# FP16
export_engine(model, ENG_FP16, half=True,  int8=False, imgsz=IMGSZ, batch=BATCH,
              workspace=WORKSPACE_GB, dynamic=False)

# INT8 (PTQ via dataset YAML; labels NOT needed for calibration)
export_engine(model, ENG_INT8, half=False, int8=True, data_yaml=DATA_YAML,
              imgsz=IMGSZ, batch=BATCH, workspace=WORKSPACE_GB,
              fraction=CALIB_FRACTION, dynamic=False)


WARNING ⚠️ TensorRT requires GPU export, automatically assigning device=0
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.8.0+cu128 CUDA:0 (Tesla T4, 15095MiB)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLO11s summary (fused): 100 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/yolo11_custom_run/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (18.3 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.64...
ONNX: export success ✅ 3.1s, saved as '/content/drive/MyDrive/yolo11_custom_run/best.onnx' (36.2 MB)

TensorRT: starting export with TensorRT 10.13.2.6...
TensorRT: input "images" with shape(1, 3, 640, 640) DataType.FLOAT
TensorRT: output "output0" with shape(1, 5, 8400) DataType.FLOAT
TensorRT: building FP32 engine as /content/drive/MyDrive/yolo11_custom_run/best.en

Fast image access ✅ (ping: 0.4±0.1 ms, read: 0.4±0.0 MB/s, size: 260.9 KB)


Scanning /content/drive/MyDrive/yolo11_custom_run/dataset/labels/val... 1508 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1508/1508 [15:39<00:00,  1.61it/s]


New cache created: /content/drive/MyDrive/yolo11_custom_run/dataset/labels/val.cache
TensorRT: input "images" with shape(1, 3, 640, 640) DataType.FLOAT
TensorRT: output "output0" with shape(1, 5, 8400) DataType.FLOAT
TensorRT: building INT8 engine as /content/drive/MyDrive/yolo11_custom_run/best.engine
TensorRT: export success ✅ 1676.1s, saved as '/content/drive/MyDrive/yolo11_custom_run/best.engine' (13.2 MB)

Export complete (1676.4s)
Results saved to /content/drive/MyDrive/yolo11_custom_run
Predict:         yolo predict task=detect model=/content/drive/MyDrive/yolo11_custom_run/best.engine imgsz=640 int8 
Validate:        yolo val task=detect model=/content/drive/MyDrive/yolo11_custom_run/best.engine imgsz=640 data=/Users/ozgurcem/yolo11_custom/custom_dataset.yaml int8 
Visualize:       https://netron.app
INT8 engine → /content/outs2/yolo11m_custom_int8_b1_640_frac1.0.engine


'/content/outs2/yolo11m_custom_int8_b1_640_frac1.0.engine'

In [ ]:
from ultralytics import YOLO
import numpy as np

def eval_map(model_id, data_yaml, imgsz=640, device=0):
    is_engine = str(model_id).endswith(".engine")
    m = YOLO(model_id, task="detect") if is_engine else YOLO(model_id)
    metrics = m.val(data=data_yaml, split="test", imgsz=imgsz, device=device, verbose=False)
    
    try:
        d = {
            "mAP50":      float(metrics.box.map50),
            "mAP50-95":   float(metrics.box.map),
            "precision":  float(metrics.box.mp),
            "recall":     float(metrics.box.mr),
            "num_images": int(getattr(metrics, "seen", 0) or getattr(metrics.box, "n", 0) or 0)
        }
    except Exception:
        d = {}
       
        for k in ("metrics/mAP50", "metrics/mAP50-95"):
            if k in metrics: d[k] = float(metrics[k])
        d.setdefault("mAP50", d.pop("metrics/mAP50", float("nan")))
        d.setdefault("mAP50-95", d.pop("metrics/mAP50-95", float("nan")))
        d.setdefault("precision", float("nan"))
        d.setdefault("recall", float("nan"))
        d.setdefault("num_images", 0)
    return d

eval_results = []
eval_results.append(("Baseline .pt",   eval_map(WEIGHTS_PT, DATA_YAML, imgsz=IMGSZ)))
eval_results.append(("TRT FP32",       eval_map(ENG_FP32,   DATA_YAML, imgsz=IMGSZ)))
eval_results.append(("TRT FP16",       eval_map(ENG_FP16,   DATA_YAML, imgsz=IMGSZ)))
eval_results.append(("TRT INT8",       eval_map(ENG_INT8,   DATA_YAML, imgsz=IMGSZ)))

print("=== Test set metrics (mAP) ===")
print(f"{'Model':14}  {'mAP50':>8}  {'mAP50-95':>10}  {'Prec':>8}  {'Recall':>8}  {'N':>6}")
for name, r in eval_results:
    print(f"{name:14}  {r['mAP50']:8.4f}  {r['mAP50-95']:10.4f}  {r['precision']:8.4f}  {r['recall']:8.4f}  {r['num_images']:6d}")


Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.8.0+cu128 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs
val: Fast image access ✅ (ping: 0.5±0.2 ms, read: 0.5±0.2 MB/s, size: 322.0 KB)


val: Scanning /content/drive/MyDrive/yolo11_custom_run/dataset/labels/test... 1008 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1008/1008 [10:03<00:00,  1.67it/s]


val: New cache created: /content/drive/MyDrive/yolo11_custom_run/dataset/labels/test.cache


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 63/63 [00:17<00:00,  3.52it/s]


                   all       1008       1008      0.969      0.869      0.917      0.391
Speed: 0.3ms preprocess, 5.5ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect/val
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.8.0+cu128 CUDA:0 (Tesla T4, 15095MiB)
Loading /content/outs2/yolo11m_custom_fp32_b1_640.engine for TensorRT inference...
Setting batch=1 input of shape (1, 3, 640, 640)
val: Fast image access ✅ (ping: 0.4±0.1 ms, read: 110.0±18.9 MB/s, size: 292.5 KB)


...

                   all       1008       1008      0.967      0.869      0.918      0.393
Speed: 0.8ms preprocess, 8.8ms inference, 0.0ms loss, 3.1ms postprocess per image
Results saved to runs/detect/val2
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.8.0+cu128 CUDA:0 (Tesla T4, 15095MiB)
Loading /content/outs2/yolo11m_custom_fp16_b1_640.engine for TensorRT inference...
Setting batch=1 input of shape (1, 3, 640, 640)
val: Fast image access ✅ (ping: 0.3±0.1 ms, read: 99.2±52.4 MB/s, size: 314.7 KB)


val: Scanning /content/drive/MyDrive/yolo11_custom_run/dataset/labels/test.cache... 1008 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1008/1008 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1008/1008 [00:20<00:00, 49.20it/s]


                   all       1008       1008      0.967      0.868      0.918      0.392
Speed: 0.8ms preprocess, 7.5ms inference, 0.0ms loss, 3.1ms postprocess per image
Results saved to runs/detect/val3
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.8.0+cu128 CUDA:0 (Tesla T4, 15095MiB)
Loading /content/outs2/yolo11m_custom_int8_b1_640_frac1.0.engine for TensorRT inference...
Setting batch=1 input of shape (1, 3, 640, 640)
val: Fast image access ✅ (ping: 0.3±0.1 ms, read: 125.6±15.0 MB/s, size: 310.7 KB)


val: Scanning /content/drive/MyDrive/yolo11_custom_run/dataset/labels/test.cache... 1008 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1008/1008 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1008/1008 [00:18<00:00, 55.47it/s]


                   all       1008       1008       0.97       0.88      0.917      0.386
Speed: 0.8ms preprocess, 5.5ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to runs/detect/val4
=== Test set metrics (mAP) ===
Model              mAP50    mAP50-95      Prec    Recall       N
Baseline .pt      0.9171      0.3914    0.9690    0.8688       0
TRT FP32          0.9178      0.3927    0.9669    0.8687       0
TRT FP16          0.9178      0.3921    0.9668    0.8675       0
TRT INT8          0.9174      0.3864    0.9703    0.8800       0


In [8]:
def resolve_split_image_paths(data_yaml, split_key="test"):
    with open(data_yaml, "r") as f:
        cfg = yaml.safe_load(f)
    root = Path(cfg.get("path", "."))
    split = cfg.get(split_key)
    assert split is not None, f"Split '{split_key}' missing in {data_yaml}"

    def collect_from_dir(d):
        return sorted([p for p in glob.glob(str(d/"**/*.*"), recursive=True)
                       if p.lower().endswith((".jpg",".jpeg",".png",".bmp",".tif",".tiff",".webp"))])

    imgs = []
    if isinstance(split, str):
        d = (root / split).resolve()
        if d.is_dir():
            imgs = collect_from_dir(d)
        else:
            imgs = sorted([p for p in glob.glob(str(d), recursive=True)
                           if p.lower().endswith((".jpg",".jpeg",".png",".bmp",".tif",".tiff",".webp"))])
    elif isinstance(split, (list, tuple)):
        for item in split:
            d = (root / item).resolve()
            imgs += collect_from_dir(d)
        imgs = sorted(set(imgs))

    return imgs

test_images = resolve_split_image_paths(DATA_YAML, "test")
assert len(test_images) > 0, "No images found in test split."
print("Test images:", len(test_images))


Test images: 1008


In [9]:
import time, numpy as np
from ultralytics import YOLO
import cv2

# Benchmark knobs
WARMUP_IMAGES     = 20       # ignore first N images when timing
LIMIT_TEST_IMAGES = None     # e.g., 300 for a quick pass; None = use all
REPEATS_PER_IMAGE = 1        # set >1 to reduce noise (keeps batch=1)
PRELOAD_IMAGES    = False    # True → load imgs into RAM (reduces disk I/O variance)

# Optional: preload to memory
preloaded = None
if PRELOAD_IMAGES:
    preloaded = []
    for p in (test_images if LIMIT_TEST_IMAGES is None else test_images[:LIMIT_TEST_IMAGES]):
        img = cv2.imread(p)  # BGR
        if img is not None:
            preloaded.append(img)
    print("Preloaded images:", len(preloaded))

def benchmark_images(model_id, img_paths, imgsz=640, conf=0.25,
                     warmup=20, limit=None, repeats=1, device=0, preloaded_imgs=None):
    """
    Returns avg_ms_per_image, fps, p50_ms, p95_ms, timed_images.
    Timings include Ultralytics pre/post (letterbox, NMS) for fairness.
    """
    is_engine = str(model_id).endswith(".engine")
    model = YOLO(model_id, task="detect") if is_engine else YOLO(model_id)

    # Build the list
    if preloaded_imgs is not None:
        # Memory mode
        imgs = preloaded_imgs
        n_total = len(imgs)
        w = min(warmup, n_total); lim = n_total if limit is None else min(limit, n_total)
        # Warmup
        for i in range(w):
            _ = model.predict(source=imgs[i], imgsz=imgsz, conf=conf, device=device, verbose=False)
        # Timed
        times = []
        for i in range(w, lim):
            for _ in range(repeats):
                t0 = time.perf_counter()
                _ = model.predict(source=imgs[i], imgsz=imgsz, conf=conf, device=device, verbose=False)
                times.append(time.perf_counter() - t0)
    else:
        # Path mode
        paths = img_paths if limit is None else img_paths[:int(limit)]
        n_total = len(paths)
        w = min(warmup, n_total)
        for p in paths[:w]:
            _ = model.predict(source=p, imgsz=imgsz, conf=conf, device=device, verbose=False)
        times = []
        for p in paths[w:]:
            for _ in range(repeats):
                t0 = time.perf_counter()
                _ = model.predict(source=p, imgsz=imgsz, conf=conf, device=device, verbose=False)
                times.append(time.perf_counter() - t0)

    if not times:
        return float("nan"), 0.0, float("nan"), float("nan"), 0

    times = np.array(times, dtype=np.float64)
    mean = float(times.mean())
    fps  = 1.0 / mean
    p50  = float(np.percentile(times, 50))
    p95  = float(np.percentile(times, 95))
    timed = len(times)
    return mean*1000.0, fps, p50*1000.0, p95*1000.0, timed

speed_results = []
for name, mid in [("Baseline .pt", WEIGHTS_PT),
                  ("TRT FP32", ENG_FP32),
                  ("TRT FP16", ENG_FP16),
                  ("TRT INT8", ENG_INT8)]:
    ms, fps, p50, p95, n = benchmark_images(
        mid, test_images, imgsz=IMGSZ, conf=CONF,
        warmup=WARMUP_IMAGES, limit=LIMIT_TEST_IMAGES,
        repeats=REPEATS_PER_IMAGE, device=0,
        preloaded_imgs=preloaded
    )
    speed_results.append((name, ms, fps, p50, p95, n))

print("\n=== Test images throughput (batch=1) ===")
print(f"{'Model':14}  {'avg ms/img':>12}  {'FPS':>9}  {'p50 ms':>9}  {'p95 ms':>9}  {'timed imgs':>12}")
for name, ms, fps, p50, p95, n in speed_results:
    print(f"{name:14}  {ms:12.2f}  {fps:9.2f}  {p50:9.2f}  {p95:9.2f}  {n:12d}")


Loading /content/outs2/yolo11m_custom_fp32_b1_640.engine for TensorRT inference...
Loading /content/outs2/yolo11m_custom_fp16_b1_640.engine for TensorRT inference...
Loading /content/outs2/yolo11m_custom_int8_b1_640_frac1.0.engine for TensorRT inference...

=== Test images throughput (batch=1) ===
Model             avg ms/img        FPS     p50 ms     p95 ms    timed imgs
Baseline .pt           26.65      37.52      23.88      39.15           988
TRT FP32               24.20      41.32      23.43      29.44           988
TRT FP16               20.87      47.92      20.18      25.55           988
TRT INT8               19.93      50.17      18.50      24.51           988


In [ ]:
import pandas as pd, shutil
DRIVE_OUT = Path("/content/drive/MyDrive/yolo11_trt_custom_results")
DRIVE_OUT.mkdir(parents=True, exist_ok=True)


for p in OUT_DIR.glob("*.engine"):
    shutil.copy2(p, DRIVE_OUT / p.name)

# Save mAP + throughput tables
eval_df = pd.DataFrame([{"model": n, **r} for n, r in eval_results])
speed_df = pd.DataFrame([{"model": n, "avg_ms_per_image": ms, "fps": fps, "p50_ms": p50, "p95_ms": p95, "timed_images": ni}
                         for n, ms, fps, p50, p95, ni in speed_results])
eval_df.to_csv(DRIVE_OUT/"test_metrics.csv", index=False)
speed_df.to_csv(DRIVE_OUT/"test_throughput.csv", index=False)

print("Saved to:", DRIVE_OUT)
for p in sorted(DRIVE_OUT.iterdir()):
    sz = os.path.getsize(p) / (1024*1024)
    print(f"{p.name:30}  {sz:6.1f} MB")


Saved to: /content/drive/MyDrive/yolo11_trt_custom_results
test_metrics.csv                   0.0 MB
test_throughput.csv                0.0 MB
yolo11m_custom_fp16_b1_640.engine    21.7 MB
yolo11m_custom_fp32_b1_640.engine    40.5 MB
yolo11m_custom_int8_b1_640_frac1.0.engine    13.2 MB
